In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pybullet as p
import pybullet_data
import time

# --- PandaSim Class Definition ---
class PandaSim:
    def __init__(self, gui=True):
        self.gui = gui
        if gui:
            # Check if connected, if not connect
            if p.getConnectionInfo()['isConnected'] == 0:
                
                p.connect(p.GUI)
                p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)
                p.resetDebugVisualizerCamera(
                    cameraDistance=1.14,      # Zoom level (smaller = closer)
                    cameraYaw=30,            # Left/Right angle (degrees)
                    cameraPitch=-30,         # Up/Down angle (degrees, negative looks down)
                    cameraTargetPosition=[0.55, -0.15, 0.] # Focus point [x, y, z]
                )
        else:
             if p.getConnectionInfo()['isConnected'] == 0:
                p.connect(p.DIRECT)
        
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.setGravity(0, 0, -9.81)
        # Load plane and robot
        if p.getNumBodies() == 0: # Avoid reloading if already loaded
            p.loadURDF("plane.urdf")
            self.robot_id = p.loadURDF("franka_panda/panda.urdf", useFixedBase=True)
        else:
            self.robot_id = 1 # Assuming index 1 if re-running

        self.arm_dof = 7
        self.ee_link_idx = 11
        self.last_ee_pos = None
        self.trace_enabled = True
        
        # Ready pose
        self.ready_pose = [0, -0.78, 0, -2.35, 0, 1.57, 0.78]
        self.reset_robot(self.ready_pose)

    def reset_robot(self, q):
        for i in range(self.arm_dof):
            p.resetJointState(self.robot_id, i, q[i])
            
    def get_joint_states(self):
        states = p.getJointStates(self.robot_id, range(self.arm_dof))
        q = np.array([s[0] for s in states])
        return q

    def get_ee_pose(self):
        state = p.getLinkState(self.robot_id, self.ee_link_idx)
        return np.array(state[0]), np.array(state[1])

    def get_jacobian(self, q):
        # Pad q with zeros for fingers to match PyBullet 9-DOF expectation
        full_q = list(q) + [0.0] * (9 - len(q))
        full_zeros = [0.0] * 9
        J_lin, J_ang = p.calculateJacobian(self.robot_id, self.ee_link_idx, [0,0,0], full_q, full_zeros, full_zeros)
        return np.array(J_lin)[:, :7], np.array(J_ang)[:, :7]
    
    def step(self):
        # Trace visualization
        current_ee_pos, _ = self.get_ee_pose()
        if self.trace_enabled and self.last_ee_pos is not None:
            p.addUserDebugLine(self.last_ee_pos, current_ee_pos, [0, 0, 1], 2.5, 0)
        self.last_ee_pos = current_ee_pos
        p.stepSimulation()
        if self.gui: time.sleep(1./240.)

    def clear_trace(self):
        p.removeAllUserDebugItems()
        self.last_ee_pos = None

# Initialize Global Simulator Instance
sim = PandaSim(gui=True)
print("Simulator Initialized.")

Simulator Initialized.


In [ ]:
def exercise_1_1():
    print("--- Exercise 1.1: Closed Form Trajectory ---")
    sim.reset_robot(sim.ready_pose)
    sim.clear_trace()
    time.sleep(1)
    
    T = 5.0
    steps = 50
    time_vec = np.linspace(0, T, steps)
    
    # Define Start and Target
    p0 = np.array([0.4, -0.2, 0.5])
    pT = np.array([0.6, 0.3, 0.3])
    
    # Visual Markers
    start_visual = p.createMultiBody(baseVisualShapeIndex=p.createVisualShape(p.GEOM_SPHERE, radius=0.01, rgbaColor=[0,1,0,1]), basePosition=p0)
    target_visual = p.createMultiBody(baseVisualShapeIndex=p.createVisualShape(p.GEOM_SPHERE, radius=0.01, rgbaColor=[1,0,0,1]), basePosition=pT)

    cartesian_trajectory = np.zeros((3, steps))

    # TODO: Define the Matrix A for the cubic polynomial constraints [x(0), v(0), x(T), v(T)]
    # Hint: x(t) = a0 + a1*t + a2*t^2 + a3*t^3
    # A = ... 
    
    for dim in range(3):
        b = np.array([p0[dim], 0, pT[dim], 0])
        
        # TODO: Solve the linear system to find coefficients
        # coeffs = ...
        
        # TODO: Evaluate polynomial at all time steps
        # cartesian_trajectory[dim, :] = ...
        pass # Remove this pass

    # Plotting
    try:
        fig, axs = plt.subplots(3, 1, figsize=(6, 8), sharex=True)
        labels = ["X", "Y", "Z"]
        for i in range(3):
            axs[i].plot(time_vec, cartesian_trajectory[i, :], 'r-', lw=2)
            axs[i].set_ylabel(labels[i])
            axs[i].grid(True)
        plt.xlabel("Time [s]")
        plt.suptitle("Generated Cubic Trajectory Profile")
        plt.show()

        # Execute on Robot
        print("Executing on robot...")
        for i in range(steps):
            target_pos = cartesian_trajectory[:, i]
            q_sol = p.calculateInverseKinematics(sim.robot_id, sim.ee_link_idx, target_pos)
            sim.reset_robot(q_sol)
            sim.step()
            time.sleep(0.05)
    except Exception as e:
        print(f"Error executing trajectory: {e}")
    finally:
        p.removeBody(target_visual)
        p.removeBody(start_visual)
        print("Done.\n")

exercise_1_1()

In [ ]:
def exercise_1_2():
    print("--- Exercise 1.2: Optimal Control ---")
    sim.reset_robot(sim.ready_pose)
    sim.clear_trace()
    time.sleep(1)
    
    dof = 7
    num_steps = 10
    start_q = np.array(sim.ready_pose)
    target_pos = np.array([0.6, 0.3, 0.2])
    
    # Visual Marker
    target_visual = p.createMultiBody(baseVisualShapeIndex=p.createVisualShape(p.GEOM_SPHERE, radius=0.03, rgbaColor=[1,0,0,1]), basePosition=target_pos)

    # Helper functions
    def unpack(x): return x[:-1].reshape((num_steps, dof)), x[-1]
    
    def get_ee_pos_wrapper(q):
        sim.reset_robot(q) 
        pos, _ = sim.get_ee_pose()
        return pos

    # --- TODO: Implement Cost Functions ---
    def cost_min_time(x):
        q, Tf = unpack(x)
        # TODO: Return cost for minimizing time
        return 0.0

    def cost_min_task_dist(x):
        q_steps, Tf = unpack(x)
        # TODO: Return cost for minimizing Cartesian path length
        return 0.0

    def cost_min_joint_dist(x):
        q_steps, Tf = unpack(x)
        # TODO: Return cost for minimizing Joint path length
        return 0.0

    # --- TODO: Implement Velocity Constraints ---
    URDF_VELOCITY_LIMIT = 2.175
    def constraint_velocity(x):
        q_steps, Tf = unpack(x)
        # TODO: Implement velocity constraint (Limit - Current_Vel >= 0)
        return 0.0

    # Optimization Setup
    x0 = np.concatenate([np.tile(start_q, num_steps), [3.0]]) 
    
    cons = [
        {'type': 'eq', 'fun': lambda x: unpack(x)[0][0] - start_q},
        {'type': 'eq', 'fun': lambda x: get_ee_pos_wrapper(unpack(x)[0][-1]) - target_pos},
        # TODO: Add your velocity constraint here
        # {'type': 'ineq', 'fun': constraint_velocity}
    ]
    bounds = [(None, None)] * (num_steps*dof) + [(0.5, 10.0)]
    
    print("Optimizing... (Please wait)")
    
    # TODO: Switch between different cost functions to test
    res = minimize(cost_min_joint_dist, x0, constraints=cons, bounds=bounds, method='SLSQP')

    if res.success:
        final_time = res.x[-1]
        print(f"Optimization Successful! Cost: {res.fun:.4f}")
        print(f"Time: {final_time:.2f}s")
        
        q_traj, _ = unpack(res.x)
        for q in q_traj:
            sim.reset_robot(q)
            sim.step()
            time.sleep(0.2)
    else:
        print("Optimization Failed:", res.message)
    
    p.removeBody(target_visual)

exercise_1_2()

--- Demo 2: Optimal Control (With Velocity Limits) ---
Optimizing... (Please wait)
Optimization Successful! Cost: 0.0286
Optimized Time to Reach Target: 3.42 seconds


In [ ]:
def exercise_1_3():
    print("--- Exercise 1.3: Replanning Under Disturbance ---")
    sim.reset_robot(sim.ready_pose)
    sim.clear_trace()
    p.removeAllUserDebugItems()
    time.sleep(1)
    
    dof = 7
    target_pos = np.array([0.6, 0.3, 0.2])
    start_q = np.array(sim.ready_pose)
    
    target_visual = p.createMultiBody(baseVisualShapeIndex=p.createVisualShape(p.GEOM_SPHERE, radius=0.03, rgbaColor=[1,0,0,1]), basePosition=target_pos)
    
    # Helper Optimizer for a segment
    def optimize_segment(start_conf, target, steps):
        x0 = np.tile(start_conf, steps).flatten()
        
        # Simple joint distance cost
        def cost(x): 
            q = x.reshape((steps, dof))
            return 0 # Change this and add the joint distance cost from previous question.
        
        def get_fk(q_flat):
            sim.reset_robot(q_flat)
            return sim.get_ee_pose()[0]

        cons = [
            {'type':'eq', 'fun': lambda x: x[:dof] - start_conf},
            {'type':'eq', 'fun': lambda x: get_fk(x[-dof:]) - target}
        ]
        res = minimize(cost, x0, constraints=cons, method='SLSQP')
        return res.x.reshape((steps, dof)) if res.success else None

    # 1. Initial Plan
    print("1. Planning Initial Path...")
    traj_initial = optimize_segment(start_q, target_pos, 20)
    
    if traj_initial is None:
        print("Initial plan failed.")
        return

    # Visualize Plan (Green)
    path_points = []
    for q in traj_initial:
        sim.reset_robot(q)
        pos, _ = sim.get_ee_pose()
        path_points.append(pos)
    for i in range(len(path_points) - 1):
        p.addUserDebugLine(path_points[i], path_points[i+1], lineColorRGB=[0, 1, 0], lineWidth=4, lifeTime=0)
    
    # Reset for Execution
    sim.reset_robot(start_q)

    # 2. Apply Disturbance
    mid_idx = 10
    q_mid = traj_initial[mid_idx].copy()
    
    print("2. Disturbance applied to elbow!")
    q_disturbed = q_mid.copy()
    q_disturbed[3] += 0.8 # Significant push
    
    # 3. TODO: Replan from Disturbed State
    print("3. Replanning...")
    # TODO: Generate a new trajectory 'traj_replan' starting from q_disturbed
    traj_replan = np.array([q_disturbed]) # Placeholder
    
    # 4. TODO: Stitch trajectories
    # full_traj = ...
    full_traj = traj_initial # Placeholder
    
    # Execute
    print("Executing Sequence...")
    for i in range(mid_idx):
        sim.reset_robot(full_traj[i])
        sim.step()
        time.sleep(0.1)
    
    sim.reset_robot(q_disturbed)
    sim.step()
    time.sleep(0.5) 
    
    # Execute Remainder
    # for i in range(mid_idx, len(full_traj)):
    #     ...

    p.removeBody(target_visual)

exercise_1_3()

In [ ]:
def exercise_1_4():
    print("--- Exercise 1.4: Jacobian Inverse Kinematics ---")
    sim.reset_robot(sim.ready_pose)
    sim.clear_trace()
    time.sleep(1)
    
    start_pos, _ = sim.get_ee_pose()
    target_pos = start_pos + np.array([0.2, 0.2, -0.2])
    
    target_visual = p.createMultiBody(baseVisualShapeIndex=p.createVisualShape(p.GEOM_SPHERE, radius=0.02, rgbaColor=[1,0,0,1]), basePosition=target_pos)
    
    dt = 0.05
    lambda_val = 0.05 # Damping factor
    q = np.array(sim.ready_pose)
    
    print(f"Chasing Target: {target_pos}")
    for i in range(300): 
        sim.reset_robot(q)
        
        curr_pos, _ = sim.get_ee_pose()
        error = target_pos - curr_pos
        
        if np.linalg.norm(error) < 0.01:
            print("Target Reached!")
            break
            
        # Get Jacobian
        J_lin, _ = sim.get_jacobian(q)
        
        # TODO: Compute J_pinv using Damped Pseudo-Inverse
        # J_pinv = ...
        J_pinv = np.zeros((7, 3)) # Placeholder
        
        # TODO: Compute joint velocity dq
        # dq = ...
        dq = np.zeros(7) # Placeholder
        
        q = q + dq * dt
        
        sim.step()
        time.sleep(0.02)
        
    p.removeBody(target_visual)

exercise_1_4()

### **End of Demo**
Run the cell below to disconnect the simulator properly.

In [15]:
p.disconnect()
print("Simulator disconnected.")

Simulator disconnected.numActiveThreads = 0
stopping threads
destroy semaphore
semaphore destroyed
Thread with taskId 0 exiting
Thread TERMINATED
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed

